In [72]:
import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
from sklearn.metrics import r2_score

In [73]:
df=pd.read_csv("city_day.csv",na_values="=")

In [74]:
df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [75]:
df.shape

(29531, 16)

In [76]:
df.dtypes

City           object
Date           object
PM2.5         float64
PM10          float64
NO            float64
NO2           float64
NOx           float64
NH3           float64
CO            float64
SO2           float64
O3            float64
Benzene       float64
Toluene       float64
Xylene        float64
AQI           float64
AQI_Bucket     object
dtype: object

In [77]:
df.isna().sum()

City              0
Date              0
PM2.5          4598
PM10          11140
NO             3582
NO2            3585
NOx            4185
NH3           10328
CO             2059
SO2            3854
O3             4022
Benzene        5623
Toluene        8041
Xylene        18109
AQI            4681
AQI_Bucket     4681
dtype: int64

In [78]:
df=df.fillna(df.mean())

<ipython-input-78-6f8c6f28e805>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df=df.fillna(df.mean())


In [79]:
label_encoder = LabelEncoder()
x=df["City"]
df["City"] = label_encoder.fit_transform(df["City"])

df["AQI_Bucket"] = label_encoder.fit_transform(df["AQI_Bucket"])

In [80]:
print("Mapping of classes to labels:")
unique_labels = set()
for city, label in zip(df["City"], x):
    y=f"{city} -> {label}"
    unique_labels.add(y)

Mapping of classes to labels:


In [81]:
print(unique_labels)

{'22 -> Shillong', '1 -> Aizawl', '21 -> Patna', '23 -> Talcher', '16 -> Jorapokhar', '15 -> Jaipur', '3 -> Amritsar', '8 -> Chennai', '19 -> Lucknow', '0 -> Ahmedabad', '11 -> Ernakulam', '7 -> Chandigarh', '5 -> Bhopal', '18 -> Kolkata', '6 -> Brajrajnagar', '13 -> Guwahati', '20 -> Mumbai', '25 -> Visakhapatnam', '10 -> Delhi', '17 -> Kochi', '12 -> Gurugram', '9 -> Coimbatore', '4 -> Bengaluru', '14 -> Hyderabad', '2 -> Amaravati', '24 -> Thiruvananthapuram'}


In [82]:
df.head(10)

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,0,2015-01-01,67.450578,118.127103,0.92000,18.22,17.15,23.483476,0.920000,27.64,133.36,0.00,0.02,0.00,166.463581,6
1,0,2015-01-02,67.450578,118.127103,0.97000,15.69,16.46,23.483476,0.970000,24.55,34.06,3.68,5.50,3.77,166.463581,6
2,0,2015-01-03,67.450578,118.127103,17.40000,19.30,29.70,23.483476,17.400000,29.07,30.70,6.80,16.40,2.25,166.463581,6
3,0,2015-01-04,67.450578,118.127103,1.70000,18.48,17.97,23.483476,1.700000,18.59,36.08,4.43,10.14,1.00,166.463581,6
4,0,2015-01-05,67.450578,118.127103,22.10000,21.42,37.76,23.483476,22.100000,39.33,39.31,7.01,18.89,2.78,166.463581,6
5,0,2015-01-06,67.450578,118.127103,45.41000,38.48,81.50,23.483476,45.410000,45.76,46.51,5.42,10.83,1.93,166.463581,6
6,0,2015-01-07,67.450578,118.127103,112.16000,40.62,130.77,23.483476,112.160000,32.28,33.47,0.00,0.00,0.00,166.463581,6
7,0,2015-01-08,67.450578,118.127103,80.87000,36.74,96.75,23.483476,80.870000,38.54,31.89,0.00,0.00,0.00,166.463581,6
8,0,2015-01-09,67.450578,118.127103,29.16000,31.00,48.00,23.483476,29.160000,58.68,25.75,0.00,0.00,0.00,166.463581,6
9,0,2015-01-10,67.450578,118.127103,17.57473,7.04,0.00,23.483476,2.248598,8.29,4.55,0.00,0.00,0.00,166.463581,6


In [83]:
df["AQI_Bucket"]=df["AQI_Bucket"].fillna(df["AQI_Bucket"].mean())

In [84]:
df["AQI_Bucket"]

0        6
1        6
2        6
3        6
4        6
        ..
29526    0
29527    3
29528    3
29529    3
29530    0
Name: AQI_Bucket, Length: 29531, dtype: int64

In [85]:
df=df.drop("Date",1)

<ipython-input-85-7d3b58cc1798>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop("Date",1)


In [86]:
x = df.drop("AQI", axis=1)
y = df["AQI"]

In [87]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [88]:
model=RandomForestRegressor(max_depth=50,random_state=0)

In [65]:
model.fit(x_train,y_train)

RandomForestRegressor(max_depth=50, random_state=0)

In [66]:
y_pred=model.predict(x_test)

In [67]:
y_pred

array([363.68      , 134.4025    , 157.81722334, ..., 333.8       ,
        60.41      , 131.32      ])

In [69]:
joblib.dump(model, 'randonforestregressormodel.pkl')

['randonforestregressormodel.pkl']

In [68]:
r2_score(y_test,y_pred)

0.9389440154254075

In [71]:
feature_importances = model.feature_importances_
print("Feature Importances:")
for feature, importance in zip(x_train.columns, feature_importances):
    print(f"{feature}: {importance:.4f}")

Feature Importances:
City: 0.0055
PM2.5: 0.4383
PM10: 0.0221
NO: 0.0214
NO2: 0.0073
NOx: 0.0074
NH3: 0.0012
CO: 0.3067
SO2: 0.0059
O3: 0.0057
Benzene: 0.0029
Toluene: 0.0040
Xylene: 0.0035
AQI_Bucket: 0.1682
